In [1]:
from flask import Flask, render_template_string, request, url_for, jsonify
import os
import numpy as np
import pandas as pd
import math


In [43]:
df = pd.read_csv("/Users/keziahdutt/Downloads/Datathon 2025/df_final.csv")
df['displayed'] = 0
df['like'] = 0
df['dislike'] = 0
df['clicked'] = 0

In [44]:
all_images = []
columns = ['simple', 'nationalism', 'reinforce beliefs', 'persuasive lang', 
           'emotional_img', 'patriotic_img', 'action_img']

for _, row in df.iterrows():
    all_images.append({
        'filename': row['img_ref'], 
        'score': [row[col] for col in columns], 
        'text': row['sequence']
    })



In [45]:
#functions
##euclidean distance
def euclidean_distance(list1, dataframe, dfrow):
    return math.sqrt((list1[0] - dataframe.loc[dfrow,'simple']) ** 2 
                     + (list1[1] - dataframe.loc[dfrow,'nationalism']) ** 2
                     + (list1[2] - dataframe.loc[dfrow,'reinforce beliefs']) ** 2
                     + (list1[3] - dataframe.loc[dfrow,'persuasive lang']) ** 2
                     + (list1[4] - dataframe.loc[dfrow,'emotional_img']) ** 2
                     + (list1[5] - dataframe.loc[dfrow,'patriotic_img']) ** 2
                     + (list1[6] - dataframe.loc[dfrow,'action_img']) ** 2)
##sample values
def select_values(neutrality_score, covmatrix, number_of_posts):
    random_values = np.random.multivariate_normal(neutrality_score, covmatrix, number_of_posts)
    return random_values
##choose the closest posts
def closest_posts(random_values, posts_dataframe):
    all_closest_images = {}
    all_closest_rows ={}
    posts_dataframe['displayed'] = 0
    for i in range(len(random_values)):
        min_distance = float('inf')
        closest = None
        closest_row = None
        for row in range(len(posts_dataframe)):
            dist = euclidean_distance(random_values[i], posts_dataframe, row)
            if dist < min_distance and posts_dataframe.loc[row,'displayed']==0 and posts_dataframe.loc[row,'like']==0 and posts_dataframe.loc[row,'dislike']==0 and posts_dataframe.loc[row,'clicked']==0:
                min_distance = dist
                closest = posts_dataframe.loc[row, 'img_ref']
                closest_row = row
        all_closest_images[closest] = min_distance
        all_closest_rows[closest_row] = min_distance
        posts_dataframe.at[closest_row, 'displayed'] = 1
    return all_closest_rows


##choose posts
def select_posts(neutrality_score, covmatrix, number_of_posts, posts_dataframe):
    val = select_values(neutrality_score, covmatrix, number_of_posts)
    result = closest_posts(val, posts_dataframe)
    sorted_result = dict(sorted(result.items(), key=lambda item: item[1]))
    sorted_list = list(sorted_result.keys())
    return sorted_list

In [59]:
#Input and tuning
##neutrality_score
#A = [.5]*7
columns = ['simple', 'nationalism', 'reinforce beliefs', 'persuasive lang', 
           'emotional_img', 'patriotic_img', 'action_img']

neutrality_score = [np.mean(df[col]) for col in columns]
##covmatrix: determine variance of MVN-distribution (the larger this is, the further of your score this will be)
B = np.diag([0.1]*7)
#A = np.diag([0.001, 0.001])


In [60]:

app = Flask(__name__)

# create lists to store liked and disliked posts
#liked_posts = set()
#disliked_posts = set()
# List of available images in the static folder
folder_path = "static"
# Get a list of all file names in the folder
available_images = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

#start page template
start_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Start Page</title>
    <link href="https://fonts.googleapis.com/css2?family=Baskervville&display=swap" rel="stylesheet">
    <style>
    body {
        display: grid;
        place-items: center;
        font-family: 'Baskervville', serif;
        background-color: #F5EFE0;
    
    }
    .start-button {
         margin-top: 30px;
            padding: 15px 30px;
            font-family: inherit;
            font-size: 30px;
            background-color: #E8DEC9;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            }
    @keyframes shrink {
        0% {transform: scale(1); }
        100% {transform: scale(0.8); }
        }
    .shrink-image {
        width: 600px;
        height: auto;
        animation: shrink 2s ease-in-out forwards;
        animation-delay: 0.3s;
    }
    </style>
</head>

<body>
    
    <img src="{{ url_for('static', filename='icons/rabbitlogo-removebg.png') }}" alt="Logo" class="shrink-image" >
    <h2 style="text-align:center;"> Welcome to Rabbit, an interactive environment <br> where you can experience World War 1 brought into the modern day. <br>
    Click start to be brought into the social media simulation. </h2>
     <form action="/home" method="get">
        <button type="submit" class="start-button">Start</button>
    </form>
</body>
</html>

"""

# Main page HTML template with clickable images and image scores
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Flask Button Example</title>
     <style>
        body {
            background-color: #F5EFE0;
            padding-top: 10px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

    ]
        .banner img {
            height: 80px;
            width: auto;
            object-fit: contain;
        }
        
        .image-text-box {
            display: flex;        
            flex-direction: column; 
            text-overflow: ellipsis;
            justify-content: flex-end;
            text-align: center;
            height:530px;
            width:400px;
            background-color: #E8DEC9;
            padding: 25px;
            font-size: 24px;
            font-family: 'Baskervville', serif;
            margin-top: 100px; 
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
            border-radius: 15px;
        }

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 400px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
        .image-wrapper {
            position:relative;
        }
    .like-btn {
        position: absolute;
        bottom: 30px; /* Position the button at the bottom */
        left: 0px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    .dislike-btn {
        position: absolute;
        bottom: 27px; /* Position the button at the bottom */
        left: 50px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    @keyframes buttonClick {
        0% {transform: scale(1);}
        50% {transform: scale(1.3);}
        100% {transform: scale(1);}
    }
   .like-btn.clicked, .dislike-btn.clicked {
    animation: buttonClick 0.3s ease-in-out;
}
</style>
        
<script>
function handleLike(postId) {
    fetch("/like", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const likeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='like']`);
            likeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            likeButton.addEventListener("animationend", () => {
                likeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}

function handleDislike(postId) {
    fetch("/dislike", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const dislikeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='dislike']`);
            dislikeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            dislikeButton.addEventListener("animationend", () => {
                dislikeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}
</script>
   
</head>

<body>
        
    <div class="banner">
        <img src="{{ url_for('static', filename='icons/rabbitlogo-removebg.png') }}" alt="Logo" >
    </div>


    <div class="image-container">

        <div class="image-wrapper">
        <form method="POST" action="/button-clicked/{{ images[0].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
                <div class="image-text-box">
                <h2> {{ images[0].text}}</h2>
                <img src="{{ url_for('static', filename=images[0].filename) }}" style="border:3px solid black; height:400px; width:400px;" alt="Click to see {{ images[0].filename }}" >
                </div>
            </button>
        </form>
   
    
        <button class="like-btn" data-post-id="{{ images[0].filename }}" data-button-type="like" onclick="handleLike('{{ images[0].filename }}')">
            <img src="{{ url_for('static', filename='icons/happyrabbit.png') }}" alt="Like" style="width: 100px; height: auto;">
         </button>
         <button class="dislike-btn" data-post-id="{{ images[0].filename }}" data-button-type="dislike" onclick="handleDislike('{{ images[0].filename }}')">
            <img src="{{ url_for('static', filename='icons/sadrabbit.png') }}" alt="Dislike" style="width: 100px; height: auto;" >
        </button>
        </div>
        
        <div class="image-wrapper">
        <form method="POST" action="/button-clicked/{{ images[1].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
            <div class="image-text-box">
                <h3> {{ images[1].text}}</h3>
                <img src="{{ url_for('static', filename=images[1].filename) }}" style="border:3px solid black; height:400px; width:400px;" alt="Click to see {{ images[1].filename }}" >
            </div>
            </button>
        </form>
        <button class="like-btn" data-post-id="{{ images[1].filename }}" data-button-type="like" onclick="handleLike('{{ images[1].filename }}')">
            <img src="{{ url_for('static', filename='icons/happyrabbit.png') }}" alt="Like" style="width: 100px; height: auto;" >
        </button>
        <button class="dislike-btn" data-post-id="{{ images[1].filename }}" data-button-type="dislike" onclick="handleDislike('{{ images[1].filename }}')">
            <img src="{{ url_for('static', filename='icons/sadrabbit.png') }}" alt="Dislike" style="width: 100px; height: auto;" >
        </button>
        </div>

    </div>

    

</body>

</html>
"""

# Template to display the clicked image
button_clicked_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Button Clicked</title>
    <style>
        body {
            background-color: #F5EFE0;
            padding-top: 5px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

    .home-button {
        background: none;  
        border: none;              
        font-size: 30px;         
        color: black;             
        cursor: pointer;          
        padding: none;               
        display: inline;           
        }   

        .home-button img {
            height: 80px;
            width: auto;
            object-fit: contain;             
        }
        
        
        
        .image-text-box {
            display: flex;        
            flex-direction: column; 
            text-overflow: ellipsis;
            justify-content: flex-end;
            text-align: center;
            height:530px;
            width:400px;
            background-color: #E8DEC9;
            padding: 25px;
            font-size: 24px;
            font-family: 'Baskervville', serif;
            margin-top: 100px; 
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
            border-radius: 15px;
        }

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 400px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
        .image-wrapper {
            position:relative;
        }

        
    .like-btn {
        position: absolute;
        bottom: 30px; /* Position the button at the bottom */
        left: 0px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    .dislike-btn {
        position: absolute;
        bottom: 27px; /* Position the button at the bottom */
        left: 50px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    @keyframes buttonClick {
        0% {transform: scale(1);}
        50% {transform: scale(1.3);}
        100% {transform: scale(1);}
    }
   .like-btn.clicked, .dislike-btn.clicked {
    animation: buttonClick 0.3s ease-in-out;
}



</style>
        
<script>
function handleLike(postId) {
    fetch("/like", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const likeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='like']`);
            likeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            likeButton.addEventListener("animationend", () => {
                likeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}

function handleDislike(postId) {
    fetch("/dislike", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const dislikeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='dislike']`);
            dislikeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            dislikeButton.addEventListener("animationend", () => {
                dislikeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}
</script>
   
</head>

<body>


    <div class="banner">
           <form action="/home" method="get">
        <button type="submit" class="home-button"> 
            <img src="{{ url_for('static', filename='icons/rabbitlogo-removebg.png') }}" alt="Click to go to home page"}}>
        </button>
    </form>
    </div>

    

<div class="image-container">

    <div class="image-text-box">
    <img src="{{ url_for('static', filename=image_filename) }}" style="border:3px solid black; height:400px; width:400px;"alt="Clicked Image">
    </div>

    <div class="image-text-box">
    <p> {{text}}</p>
    </div>
    
</div>

    
</body>
</html>
"""


@app.route("/", methods=["GET"])
def start():
        return render_template_string(start_template)

@app.route("/home", methods=["GET"])
def home():
    global neutrality_score
    res = select_posts(neutrality_score,B,2,df)
    selected_images = [all_images[res[0]], all_images[res[1]]]
    return render_template_string(html_template, neutrality_score=neutrality_score, images=selected_images)


@app.route("/button-clicked/<image_filename>", methods=["POST"])
def button_clicked(image_filename):
    global neutrality_score
    
    # Recalculate the image list (just like in the home route) to get the image score
    #nr_images = len(available_images)
    #images = []
    #for idx, filename in enumerate(available_images):
        #image_score = 
        #images.append({'filename': filename, 'score': round(image_score, 2)})  # Store both filename and score
    
    # Find the image with the corresponding filename
    image = next(image for image in all_images if image['filename'] == image_filename)
    image_score = image['score']  # Get the image score
    text = image['text']
    
    return render_template_string(button_clicked_template, neutrality_score=neutrality_score, image_filename=image_filename, image_score=image_score, text=text)
def update_csv(img_ref, action):
    # Update the respective column
    if action == "like":
        df.loc[df["img_ref"] == img_ref, "like"] += 1
    elif action == "dislike":
        df.loc[df["img_ref"] == img_ref, "dislike"] += 1

    #df.to_csv(CSV_FILE, index=False)
    return True, df






@app.route("/like", methods=["POST"])
def like():
    global neutrality_score
    post_id = request.json.get("img_ref")  # Should match the 'img_ref' column from CSV

    try:
        post_index = str(post_id)  # Assuming post_id is an index (adjust as needed)
        neutrality_score[post_index] = min(1.0, neutrality_score[post_index] + 0.1)
        return jsonify({"success": True, "neutrality_score": neutrality_score})
    except (IndexError, ValueError):
        return jsonify({"success": False, "message": "Invalid post ID"}), 400

@app.route("/dislike", methods=["POST"])
def dislike():
    global neutrality_score
    post_id = request.json.get("img_ref")

    try:
        post_index = str(post_id)
        neutrality_score[post_index] = max(0.0, neutrality_score[post_index] - 0.1)
        return jsonify({"success": True, "neutrality_score": neutrality_score})
    except (IndexError, ValueError):
        return jsonify({"success": False, "message": "Invalid post ID"}), 400




    return jsonify({"success": True, "neutrality_score": neutrality_score})
# Run the Flask app
from werkzeug.serving import run_simple

if __name__ == "__main__":
    run_simple("localhost", 5001, app)

 * Running on http://localhost:5001
Press CTRL+C to quit
127.0.0.1 - - [17/Feb/2025 16:25:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 16:25:13] "GET /static/icons/rabbitlogo-removebg.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 16:25:15] "GET /home HTTP/1.1" 200 -
127.0.0.1 - - [17/Feb/2025 16:25:15] "GET /static/icons/rabbitlogo-removebg.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 16:25:15] "GET /static/SdhfaofteattfTbaSabi.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 16:25:15] "GET /static/icons/happyrabbit.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 16:25:15] "GET /static/VioTiAoGvLhtiHactmDA.png HTTP/1.1" 304 -
127.0.0.1 - - [17/Feb/2025 16:25:15] "GET /static/icons/sadrabbit.png HTTP/1.1" 304 -
[2025-02-17 16:25:19,988] ERROR in app: Exception on /like [POST]
Traceback (most recent call last):
  File "/Users/keziahdutt/Downloads/Datathon 2025/rabbitenv/lib/python3.10/site-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()

In [ ]:

@app.route("/like", methods=["POST"])
def like():
    global neutrality_score
    post_id = request.json.get("img_ref")
    liked_posts.add(post_id)
    success, neutrality_score = update_csv(post_id, "like")
    neutrality_score = min(1.0, neutrality_score + 0.1)
    return jsonify({"success": success, "neutrality_score": neutrality_score})


@app.route("/dislike", methods=["POST"])
def dislike():
    global neutrality_score
    post_id = request.json.get("post_id")
    disliked_posts.add(post_id)
    neutrality_score = max(0.0, neutrality_score - 0.1)

In [ ]:

app = Flask(__name__)

# Initialize the neutrality score at 0.5
#neutrality_score = 0.5
# create lists to store liked and disliked posts
liked_posts = set()
disliked_posts = set()
# List of available images in the static folder
folder_path = "static"
# Get a list of all file names in the folder
available_images = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]

#start page template
start_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Start Page</title>
    <link href="https://fonts.googleapis.com/css2?family=Baskervville&display=swap" rel="stylesheet">
    <style>
    body {
        display: grid;
        place-items: center;
        font-family: 'Baskervville', serif;
        background-color: #F5EFE0;
    
    }
    .start-button {
         margin-top: 30px;
            padding: 15px 30px;
            font-family: inherit;
            font-size: 30px;
            background-color: #E8DEC9;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            }
    @keyframes shrink {
        0% {transform: scale(1); }
        100% {transform: scale(0.8); }
        }
    .shrink-image {
        width: 600px;
        height: auto;
        animation: shrink 2s ease-in-out forwards;
        animation-delay: 0.3s;
    }
    </style>
</head>

<body>
    
    <img src="{{ url_for('static', filename='icons/rabbitlogo-removebg.png') }}" alt="Logo" class="shrink-image" >
    <h2 style="text-align:center;"> Welcome to Rabbit, an interactive environment <br> where you can experience World War 1 brought into the modern day. <br>
    Click start to be brought into the social media simulation. </h2>
     <form action="/home" method="get">
        <button type="submit" class="start-button">Start</button>
    </form>
</body>
</html>

"""

# Main page HTML template with clickable images and image scores
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Flask Button Example</title>
     <style>
        body {
            background-color: #F5EFE0;
            padding-top: 10px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

    ]
        .banner img {
            height: 80px;
            width: auto;
            object-fit: contain;
        }
        
        .image-text-box {
            display: flex;        
            flex-direction: column; 
            text-overflow: ellipsis;
            justify-content: flex-end;
            text-align: center;
            height:530px;
            width:400px;
            background-color: #E8DEC9;
            padding: 25px;
            font-size: 24px;
            font-family: 'Baskervville', serif;
            margin-top: 100px; 
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
            border-radius: 15px;
        }

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 400px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
        .image-wrapper {
            position:relative;
        }
    .like-btn {
        position: absolute;
        bottom: 30px; /* Position the button at the bottom */
        left: 0px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    .dislike-btn {
        position: absolute;
        bottom: 27px; /* Position the button at the bottom */
        left: 50px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    @keyframes buttonClick {
        0% {transform: scale(1);}
        50% {transform: scale(1.3);}
        100% {transform: scale(1);}
    }
   .like-btn.clicked, .dislike-btn.clicked {
    animation: buttonClick 0.3s ease-in-out;
}
</style>
        
<script>
function handleLike(postId) {
    fetch("/like", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const likeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='like']`);
            likeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            likeButton.addEventListener("animationend", () => {
                likeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}

function handleDislike(postId) {
    fetch("/dislike", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const dislikeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='dislike']`);
            dislikeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            dislikeButton.addEventListener("animationend", () => {
                dislikeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}
</script>
   
</head>

<body>
        
    <div class="banner">
        <img src="{{ url_for('static', filename='icons/rabbitlogo-removebg.png') }}" alt="Logo" >
    </div>


    <div class="image-container">

        <div class="image-wrapper">
        <form method="POST" action="/button-clicked/{{ images[0].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
                <div class="image-text-box">
                <h2> {{ images[0].text}}</h2>
                <img src="{{ url_for('static', filename=images[0].filename) }}" style="border:3px solid black; height:400px; width:400px;" alt="Click to see {{ images[0].filename }}" >
                </div>
            </button>
        </form>
   
    
        <button class="like-btn" data-post-id="{{ images[0].filename }}" data-button-type="like" onclick="handleLike('{{ images[0].filename }}')">
            <img src="{{ url_for('static', filename='icons/happyrabbit.png') }}" alt="Like" style="width: 100px; height: auto;">
         </button>
         <button class="dislike-btn" data-post-id="{{ images[0].filename }}" data-button-type="dislike" onclick="handleDislike('{{ images[0].filename }}')">
            <img src="{{ url_for('static', filename='icons/sadrabbit.png') }}" alt="Dislike" style="width: 100px; height: auto;" >
        </button>
        </div>
        
        <div class="image-wrapper">
        <form method="POST" action="/button-clicked/{{ images[1].filename }}" style="display: inline;">
            <button type="submit" style="border: none; background: none; padding: 0;">
            <div class="image-text-box">
                <h3> {{ images[1].text}}</h3>
                <img src="{{ url_for('static', filename=images[1].filename) }}" style="border:3px solid black; height:400px; width:400px;" alt="Click to see {{ images[1].filename }}" >
            </div>
            </button>
        </form>
        <button class="like-btn" data-post-id="{{ images[1].filename }}" data-button-type="like" onclick="handleLike('{{ images[1].filename }}')">
            <img src="{{ url_for('static', filename='icons/happyrabbit.png') }}" alt="Like" style="width: 100px; height: auto;" >
        </button>
        <button class="dislike-btn" data-post-id="{{ images[1].filename }}" data-button-type="dislike" onclick="handleDislike('{{ images[1].filename }}')">
            <img src="{{ url_for('static', filename='icons/sadrabbit.png') }}" alt="Dislike" style="width: 100px; height: auto;" >
        </button>
        </div>

    </div>

    

</body>

</html>
"""

# Template to display the clicked image
button_clicked_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Button Clicked</title>
    <style>
        body {
            background-color: #F5EFE0;
            padding-top: 5px;
        }

        .banner {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 80px; /* Adjust height as needed */
            background-color: #E8DEC9; /* A slightly darker shade for contrast */
            display: flex;
            padding: 10px 20px;
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
        }

    .home-button {
        background: none;  
        border: none;              
        font-size: 30px;         
        color: black;             
        cursor: pointer;          
        padding: none;               
        display: inline;           
        }   

        .home-button img {
            height: 80px;
            width: auto;
            object-fit: contain;             
        }
        
        
        
        .image-text-box {
            display: flex;        
            flex-direction: column; 
            text-overflow: ellipsis;
            justify-content: flex-end;
            text-align: center;
            height:530px;
            width:400px;
            background-color: #E8DEC9;
            padding: 25px;
            font-size: 24px;
            font-family: 'Baskervville', serif;
            margin-top: 100px; 
            box-shadow: 0px 4px 8px rgba(0, 0, 0, 0.1);
            border-radius: 15px;
        }

        .image-container {
            display: flex;
            justify-content: center; /* Centers horizontally */
            align-items: center; /* Aligns items in the middle */
            gap: 20px; /* Adds spacing between images */
            margin-top: 200px; /* Adds some space from the top */
        
        }
        .image-container img {
            width: 400px;
            height: auto;
            display: block;
            border-radius: 15px;
        }
        .image-wrapper {
            position:relative;
        }

        
    .like-btn {
        position: absolute;
        bottom: 30px; /* Position the button at the bottom */
        left: 0px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    .dislike-btn {
        position: absolute;
        bottom: 27px; /* Position the button at the bottom */
        left: 50px; /* Position the button to the left */
        background: none; /* Remove background color */
        border: none; /* Remove border */
        padding: 0; /* Remove padding */
        margin: 0; /* Remove margin (if needed) */
        cursor: pointer; /* Change cursor to pointer on hover */
    }
    @keyframes buttonClick {
        0% {transform: scale(1);}
        50% {transform: scale(1.3);}
        100% {transform: scale(1);}
    }
   .like-btn.clicked, .dislike-btn.clicked {
    animation: buttonClick 0.3s ease-in-out;
}



</style>
        
<script>
function handleLike(postId) {
    fetch("/like", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const likeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='like']`);
            likeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            likeButton.addEventListener("animationend", () => {
                likeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}

function handleDislike(postId) {
    fetch("/dislike", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ post_id: postId })
    })
    .then(response => response.json())
    .then(data => {
        if (data.success) {
            const dislikeButton = document.querySelector(`button[data-post-id='${postId}'][data-button-type='dislike']`);
            dislikeButton.classList.add("clicked");  // Add the "clicked" class
            document.getElementById("neutrality-score").innerText = data.neutrality_score.toFixed(1);

            // Remove the "clicked" class after the animation ends
            dislikeButton.addEventListener("animationend", () => {
                dislikeButton.classList.remove("clicked");
            }, { once: true });
        } else {
            alert(data.message);
        }
    })
    .catch(error => console.error("Error:", error));
}
</script>
   
</head>

<body>


    <div class="banner">
           <form action="/home" method="get">
        <button type="submit" class="home-button"> 
            <img src="{{ url_for('static', filename='icons/rabbitlogo-removebg.png') }}" alt="Click to go to home page"}}>
        </button>
    </form>
    </div>

    

<div class="image-container">

    <div class="image-text-box">
    <img src="{{ url_for('static', filename=image_filename) }}" style="border:3px solid black; height:400px; width:400px;"alt="Clicked Image">
    </div>

    <div class="image-text-box">
    <p> {{text}}</p>
    </div>
    
</div>

    
</body>
</html>
"""


@app.route("/", methods=["GET"])
def start():
        return render_template_string(start_template)

@app.route("/home", methods=["GET"])
def home():
    global neutrality_score
    res = select_posts(neutrality_score,B,2,df)
    selected_images = [all_images[res[0]], all_images[res[1]]]
    return render_template_string(html_template, neutrality_score=neutrality_score, images=selected_images)


@app.route("/button-clicked/<image_filename>", methods=["POST"])
def button_clicked(image_filename):
    global neutrality_score
    
    # Recalculate the image list (just like in the home route) to get the image score
    #nr_images = len(available_images)
    #images = []
    #for idx, filename in enumerate(available_images):
        #image_score = 
        #images.append({'filename': filename, 'score': round(image_score, 2)})  # Store both filename and score
    
    # Find the image with the corresponding filename
    image = next(image for image in images if image['filename'] == image_filename)
    image_score = image['score']  # Get the image score
    text = image['text']
    
    return render_template_string(button_clicked_template, neutrality_score=neutrality_score, image_filename=image_filename, image_score=image_score, text=text)

@app.route("/like", methods=["POST"])
def like():
    global neutrality_score
    post_id = request.json.get("post_id")
    liked_posts.add(post_id)
    neutrality_score = min(1.0, neutrality_score + 0.1)
    return jsonify({"success": True, "neutrality_score": neutrality_score})


@app.route("/dislike", methods=["POST"])
def dislike():
    global neutrality_score
    post_id = request.json.get("post_id")
    disliked_posts.add(post_id)
    neutrality_score = max(0.0, neutrality_score - 0.1)

    return jsonify({"success": True, "neutrality_score": neutrality_score})
# Run the Flask app
from werkzeug.serving import run_simple

if __name__ == "__main__":
    run_simple("localhost", 5001, app)